![logo](https://resolvephilly.org/themes/custom/resolvephl-ci/logo.svg)

# Scraping and standardizing Pennsylvania Act 146 Individual Reports

**Author:** Julie Christie | Director of Data & Impact

**Partnering Team:** Our Kids

**Date:** March 28, 2024

## Background

Pennsylvania publishes reports on child fatalities and near fatalities as a result of child abuse. The reports include the age, sex, county, and date of the incident, as well as whether the family was known to the local department of human/family services in the 16 months preceding the incident. The detailed reports are organized online in a way that can be scraped for some, but not all of the information. The first page of the `.pdf` reports contain the rest of the necessary information.

### Goal of Analysis

Specifically, Resolve is looking to understand the frequency at which children who experience abuse that results in their death/near death are already known to the system. We are exploring these rates at the county level to understand what the statewide trend is, and whether Philadelphia exceeds that trend.

### Glossary

-   **Act 146** -- *"Act 146 of 2006 went into effect on May 8, 2007. A major provision of this law requires that the department prepare a non-identifying summary for the governor and the General Assembly of findings for each case of substantiated child abuse or neglect that has resulted in a child fatality or near fatality."*
-   **Near fatality** -- *Definition TKTK*
-   **DA De-certification** -- *This gets assigned to a report when the District Attorney determines that the incident was not a result of child abuse.*

### Data

-   [Child Fatality/Near Fatality Reports](https://www.dhs.pa.gov/docs/OCYF/Pages/Fatality-Reports.aspx) --- A table of the reports with some information and URLs to the actual reports. | No metadata available

### Tools

-   [Python](python.org) -- *Base code to facilitate scraping*
-   [Pandas](https://pandas.pydata.org/) -- *More robust data anlysis*
-   [Regex](https://developers.google.com/edu/python/regular-expressions) -- *Regular Expressions, or Regex, to parse out patterns of characters*
-   [PDF Plumber](https://github.com/jsvine/pdfplumber) -- *Parse information from .pdf files*
-   [Web Scraper](https://www.webscraper.io/documentation?utm_source=extension&utm_medium=popup) -- *Scrape data from a webpage*
-   [Excel](https://www.microsoft.com/en-us/microsoft-365/p/excel/cfq7ttc0hr4r?activetab=pivot:overviewtab) -- *Clean and analyze tabulated data*

### Limitations
- A "certifying pysician" makes an individual call on whether a child's death/near death is the result of abuse, meaning that human error may result in cases not being documented in these reports
- Child fatalities and near fatalities as a result of abuse are an incredibly small and extreme subset of the overall abuse that children face.
- Counties not recorded in scraped data pre-2016; 764 cases need counties assigned
- 922 Reports have URLs that indicate they have been de-certified and therefore have no report details. These all share one of 6 common URLs:
1. https://www.dhs.pa.gov/docs/OCYF/Pages/DA-Certification.aspx
2. https://auth-agency.pa.egov.com/sites/HumanServices/docs/OCYF/Pages/Unlinked-Report.aspx
3. https://www.dhs.pa.gov/docs/OCYF/Pages/Unlinked-Report.aspx
4. https://auth-agency.pa.egov.com/sites/HumanServices/docs/OCYF/Pages/DA-Certification.aspx
5. https://www.dhs.pa.gov/docs/OCYF/Pages/Decertification.aspx
6. https://auth-agency.pa.egov.com/sites/HumanServices/docs/OCYF/Pages/Decertification.aspx

## Cleaning

### Scrape reports

1. Use Web Scraper to extract report URLs and associated data into a `.csv` file. This creates a new row of data for each report, and each column respectively. The sitemap is:

```
{"_id":"pa_33_reports","startUrl":["https://www.dhs.pa.gov/docs/OCYF/Pages/Fatality-Reports.aspx"],"selectors":[{"clickActionType":"real","clickElementSelector":"#ctl00_ctl49_g_2f5406ff_66ab_4f5d_b7c7_f984206b2ab7_ddlPageSizer > option:nth-child(5)","clickElementUniquenessType":"uniqueText","clickType":"clickOnce","delay":2000,"discardInitialElements":"do-not-discard","id":"select_all","multiple":false,"parentSelectors":["_root"],"selector":"_parent_","type":"SelectorElementClick"},{"id":"report","multiple":true,"parentSelectors":["select_all"],"selector":"tr:nth-of-type(n+2)","type":"SelectorElement"},{"id":"info","multiple":false,"parentSelectors":["report"],"regex":"","selector":"td:nth-of-type(1)","type":"SelectorText"},{"id":"report_id","multiple":false,"parentSelectors":["report"],"regex":"","selector":"td:nth-of-type(2)","type":"SelectorText"},{"id":"fatality","multiple":false,"parentSelectors":["report"],"regex":"","selector":"td:nth-of-type(3)","type":"SelectorText"},{"id":"year","multiple":false,"parentSelectors":["report"],"regex":"","selector":"td:nth-of-type(4)","type":"SelectorText"},{"id":"date","multiple":false,"parentSelectors":["report"],"regex":"","selector":"td:nth-of-type(5)","type":"SelectorText"},{"id":"DA_Cert","multiple":false,"parentSelectors":["report"],"regex":"","selector":"td:nth-of-type(6)","type":"SelectorText"},{"id":"report_url","linkType":"linkFromHref","multiple":false,"parentSelectors":["report"],"selector":"a","type":"SelectorLink"}]}
```

The scraped data was dirtly due to poor organization and requires further cleaning in Excel.

2. Remove unnecessary column for "source URL" because the values were identical for all the data.
3. Renamed column "gender" to `Sex` because the actual data reflected sex, not gender identity.
4. Use text-to-columns function to expand "info" column on the colon (:) to become `Region`, `County`, `Sex`, and `Age` and manually cleaned any mistakes/bad parsing.
5. Created a new column `further_info` to identify reports that did not have values for `County`, `Sex`, or `Age` and would require further searching in reports to fill out.
6. Aggregate all ages to be in units of 1 year. All stated ages from "0 days" to "12 months" was recoded as "<1". Ages listed as "12 months" to "24 months" recoded to "1". Then removed the word "year" after age because all are now in the same unit.
7. Clean the `Date` column (for more detail, please reach out for data diary records
   1. Identify records where there are missing values and require further information from the report. (764 records)
   2. Extract dates from the `report_title` column with formula: `=TEXTJOIN("",TRUE,IFERROR(MID([@[report_title_copy]],ROW(INDIRECT("1:"&LEN([@[report_title_copy]]))),1)*1,""))`
   3. Check lenghth of extracted dates to find and then manually fix incorrect dates using formula `=IF(LEN([@[extracted_date]])>6, "NO", "Date")`
   4. Convert the values of the extracted dates into a readable date format with formula `=DATE(20&RIGHT([@[date_num]],2), LEFT([@[date_num]],2), MID([@[date_num]],3,2))`
   5. Input new dates into the `Date` column
   6. Many dates defaulted to have the year 2024 but the `Year` column did not match (835 records). Parsed month and date together with correct year with formula `=DATE([@year], TEXT([@date],"MM"), TEXT([@date],"DD"))` and then replaced with the value in `Date` column
8. Moved misaligned values back into the `DA_Cert` column
9. Ranamed column "report_url" to `report_title` to be more intuitively named

### Parse single report

In [3]:
pip install pdfplumber

  Using cached pdfplumber-0.11.0-py3-none-any.whl.metadata (39 kB)
  Using cached pdfminer.six-20231228-py3-none-any.whl.metadata (4.2 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.8 MB/s eta 0:00:00
  Using cached charset_normalizer-3.3.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (33 kB)
  Using cached cryptography-42.0.7-cp39-abi3-macosx_10_12_universal2.whl.metadata (5.3 kB)
  Using cached cffi-1.16.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (1.5 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
Using cached pdfplumber-0.11.0-py3-none-any.whl (56 kB)
Using cached pdfminer.six-20231228-py3-none-any.whl (5.6 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 5.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 5.6 MB/s eta 0:00:00a 0:00:01
Using cached charset_normalizer-3.3.2-cp312-cp312-macosx_11_0_arm64.whl (119 kB)
Using cached cryptography-42.0.7-cp39-abi3-macosx_10_12_universal2.whl (5.9 MB)
Usi

In [2]:
pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 427.9 kB/s eta 0:00:00a 0:00:01
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 4.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 6.6 MB/s eta 0:00:0000:0100:01
Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)
Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)
Note: you may need to restart the kernel to use updated packages.


1. Import the needed libraries

In [1]:
import re #regular expressions

import pdfplumber
import pandas as pd
from collections import namedtuple

2. Create the column names for the data that you are extracting from the pdf. `known` refers to whether the family was known to county DHS in the 16 months before the incident, and `county` identifies which county that was.

In [ ]:
Line = namedtuple('Line', 'known county')

3. Create a regular expressions function to parse out `known` and `county`

In [ ]:
known_re = re.compile(
    
)